In [ ]:
# Cell 1：MapAnything（Colab）在线模型推理 + 本机图片上传 + 下载结果
#
# 目标：完全在 Google Colab 云端运行推理：
# - 模型：从 HuggingFace 在线加载（不使用你本地的离线模型文件）
# - 图片：从你本机上传到 Colab（或你也可以改为 Drive 路径）
# - 结果：在 Colab 生成后打包下载到你本机（你再保存到任意本地路径）
#
# 说明：
# - 本 Notebook 调用 scripts/demo_images_only_inference.py 进行推理
# - 在线加载模型需要 Colab 能访问网络（HuggingFace）
# - 8GB 显存建议：--memory_efficient_inference + --max_views/--stride

In [ ]:
# Cell 2：Colab 环境准备（可直接运行）
!nvidia-smi || true

# 方式 A（推荐）：用你自己的 fork/仓库（确保包含你本地改动，比如 demo_images_only_inference.py 支持 --checkpoint_path）
# 将下面地址替换成你的 fork，或把仓库上传到 Drive 再 %cd 到对应目录。
REPO_URL = "https://github.com/facebookresearch/map-anything.git"  # TODO: 换成你的 fork（如果你改过脚本）

%cd /content
!rm -rf map-anything
!git clone $REPO_URL
%cd /content/map-anything

# 安装（推理最小依赖）
!pip -q install -U pip
!pip -q install -e .

In [ ]:
# Cell 3：挂载 Google Drive（可选，但离线模型/图片一般都放 Drive）
from google.colab import drive
drive.mount("/content/drive")

# TODO：按你的实际路径修改：
IMAGE_FOLDER = "/content/drive/MyDrive/your_images_folder"
CKPT_PATH = "/content/drive/MyDrive/checkpoints/model.safetensors"
CONFIG_JSON_PATH = "/content/drive/MyDrive/checkpoints/config.json"

# 输出必须是一个文件名（建议 .glb）
OUTPUT_GLB = "/content/drive/MyDrive/mapanything_outputs/output.glb"

In [ ]:
# Cell 4：在 Colab 里生成一个“等价于 demo_images_only_inference.py”的脚本（在线模型）
#
# 说明：VS Code 本地 Notebook 的 Python kernel 可能没有 IPython 的 `%%writefile` magic。
# 这里改为用纯 Python 写文件，兼容 Colab/VS Code/普通 Python。

from pathlib import Path

out_file = Path("demo_images_only_inference_colab.py")
script = '''import argparse
import os
from pathlib import Path

import numpy as np
import torch

from mapanything.models import MapAnything
from mapanything.utils.image import load_images
from mapanything.utils.viz import predictions_to_glb


def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="MapAnything images-only inference (Colab inline script)")
    p.add_argument("--image_folder", type=str, required=True)
    p.add_argument("--model", type=str, default=None, help="HF repo id or local dir. Default: facebook/map-anything")
    p.add_argument("--apache", action="store_true", help="Use facebook/map-anything-apache")
    p.add_argument("--memory_efficient_inference", action="store_true", default=False)
    p.add_argument("--stride", type=int, default=1, help="Load every nth image")
    p.add_argument("--max_views", type=int, default=None, help="Limit number of views")
    p.add_argument("--no_amp", action="store_true", default=False)
    p.add_argument("--amp_dtype", type=str, default="bf16", choices=["bf16", "fp16", "fp32"])
    p.add_argument("--save_glb", action="store_true", default=False)
    p.add_argument("--output_path", type=str, default="output.glb")
    return p


def main() -> None:
    args = build_parser().parse_args()

    os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    if args.model is None:
        model_id = "facebook/map-anything-apache" if args.apache else "facebook/map-anything"
    else:
        model_id = args.model
    print("Loading model:", model_id)
    model = MapAnything.from_pretrained(model_id).to(device)

    stride = max(int(args.stride), 1)
    print("Loading images from:", args.image_folder, "(stride=", stride, ")")
    views = load_images(args.image_folder, stride=stride)
    if args.max_views is not None:
        views = views[: max(int(args.max_views), 1)]
    print("Loaded", len(views), "views")

    print("Running inference...")
    outputs = model.infer(
        views,
        memory_efficient_inference=bool(args.memory_efficient_inference),
        use_amp=not args.no_amp,
        amp_dtype=args.amp_dtype,
        apply_mask=True,
        mask_edges=True,
    )
    print("Inference complete!")

    if not args.save_glb:
        print("Skipping GLB export (--save_glb not specified)")
        return

    out_path = Path(args.output_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    world_points_list = []
    images_list = []
    masks_list = []
    for pred in outputs:
        pts3d_np = pred["pts3d"][0].detach().cpu().numpy()
        image_np = pred["img_no_norm"][0].detach().cpu().numpy()
        mask = pred["mask"][0].squeeze(-1).detach().cpu().numpy().astype(bool)
        world_points_list.append(pts3d_np)
        images_list.append(image_np)
        masks_list.append(mask)

    predictions = {
        "world_points": np.stack(world_points_list, axis=0),
        "images": np.stack(images_list, axis=0),
        "final_masks": np.stack(masks_list, axis=0),
    }

    print("Exporting GLB to:", str(out_path))
    scene_3d = predictions_to_glb(predictions, as_mesh=True)
    scene_3d.export(str(out_path))
    print("Done:", str(out_path))


if __name__ == "__main__":
    main()
'''

out_file.write_text(script, encoding="utf-8")
print("Wrote:", str(out_file.resolve()))

UsageError: Line magic function `%%writefile` not found.


In [ ]:
# Cell 5：调用上一个 Cell 生成的脚本运行推理，然后下载结果到本机
import os
from pathlib import Path
from google.colab import files

# 在线加载模型：facebook/map-anything（默认 CC-BY-NC）；如果你需要 Apache 许可可换成 facebook/map-anything-apache
MODEL_ID = "facebook/map-anything"

# 输出到 Colab 本地文件系统，随后下载到你本机
OUT_DIR = Path("/content/mapanything_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_GLB = str(OUT_DIR / "output.glb")

# 8GB 显存建议参数（你可以逐步调大 max_views 或减小 stride）
MAX_VIEWS = 30
STRIDE = 2
AMP_DTYPE = "fp16"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

!python demo_images_only_inference_colab.py \
  --image_folder "$IMAGE_FOLDER" \
  --model "$MODEL_ID" \
  --memory_efficient_inference \
  --amp_dtype "$AMP_DTYPE" \
  --stride $STRIDE \
  --max_views $MAX_VIEWS \
  --save_glb \
  --output_path "$OUTPUT_GLB"

# 下载到本机（浏览器会提示保存；你可选择任意本地路径）
files.download(OUTPUT_GLB)